# Resumen
El objetivo de este proyecto es predecir la cotización bursátil de las empresas del SP500. Para ello se parte de la premisa que, pese a que los mercados representan correctamente el valor de una empresa, existen correciones a lo largo del tiempo y entre distintas empresas y sectores. El objetivo es obtener una generalización de el valor de una empresa utilizando las valoraciones históricas del mercado con el fin de predecir si una empresa esta en la norma histórica o si en cambio esta sobrevalorada o infravalorada.
<br>

El proyecto se ha abordado de forma secuencial y se ha dividio en cuatro fases:
1. Una primera fase de **Recolección de Datos**
2. Una segunda fase de **Limpieza**
3. Una tercera de **EDA + Generación de variables**
4. Una cuarta de **Modelado**

## Recolección y Limpieza
Futuros desarrollos
- Crear un script con el que parsear nosotros directamente los datos de la SEC, ya que la API empleada no disponía de los datos de muchas empresas
- Crear una base de datos SQL con las tablas limpias.

### gather.ipynb
El principal objetivo de esta fase era obtener el dataset de entrada para la generación de variables usadas en el modelo. Las fuentes de datos han sido:
- <a href="https://en.wikipedia.org/wiki/List_of_S%26P_500_companies">Listado de empresas que perctenecen al SP500</a>
- <a href="https://www.followingthetrend.com/trading-evolved/">.csv del libro Trading Evolved, de Andreas F. Clenow</a>. que incluye un time series que, para cada fecha, los símbolos de todas las empresas constituyentes del índice<a href="https://www.dropbox.com/s/etocgt9zgeedo22/data.zip?dl=0&file_subpath=%2Fdata%2Findex_members">link al dataset</a>.
- <a href="https://www.sec.gov/files/company_tickers.json">JSON</a> que mapea el símbolo de cada empresa con su CIK, que es el identificador que le asigna la <a href="https://www.sec.gov/edgar/searchedgar/companysearch.html">Securities And Exchange Commission</a> de EEUU a cada empresa cotizada.
- También de la API de la SEC se han obtenido un dataframe con la información sobre los estados financieros de cada compañia (a partir de su CIK)
- Utilizando <a href="https://github.com/mortada/fredapi">fredapi</a> se ha atacado la API de la <a href="https://fred.stlouisfed.org/docs/api/fred/">Federal Reserve of Saint Louis</a> con el fin de obtener datos historicos de la coyuntura económica en EEUU.
- Para obtener los estados financieros anuales (Balance Sheet, Income Statement & Cash Flow Statement), así como las cotización bursátil diaria de cada compañia, se ha atacado <a href="https://site.financialmodelingprep.com/developer/docs#Company-Financial-Statements">esta API</a>.

### clean.ipynb
El output del proceso de limpieza son cinco tablas:
1. Una tabla de **compañias**, cuya clave primaria es el identificador CIK de cada empresa, e incluye información sobre su nombre, sector, simbolo con el que cotiza, etc.
2. Una tabla de **estancias en el índice**, sin clave primaria, en la que cada observación representa la estancia de una empresa en el índice, de forma que, si una empresa **A**, perteneció al SP500 de 1999 a 2003, y de 2007 a 2015, habrá dos observaciones, una para cada estancia.
3. Una tabla con **datos de coyuntura económica** para cada fecha
4. Una tabla con la **cotización bursátil** de cada símbolo cada día.
5. Una tabla **de entrada al modelo** con los estados financieros, unidos a todas las variables de coyuntura económica en la fecha de publicación de los estados, así como la variable **target**.

## EDA + Generación de Variables
### feature_engineering.ipynb y EDA.ipynb
En primer lugar se ha <a href="https://en.wikipedia.org/wiki/Winsorizing">Winsorizado</a> las variables (ya que había valores muy extremos debidos a errores de medida) y se ha aplicado una transformación <a href="https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html">Yeo-Johnson</a> con el fin de normalizar los datos.

Posteriormente, se probaron por encima un modelo de regresión lineal, con los parametros de entrada del modelo en valores absolutos (`myvars.py`) y un modelo no lineal (gradient boosting) con el objetivo de opbtener una primera intuición de qué variables son las más importantes para predecir el target. Se obtuvo un mse de 1846.Se obtuvo que las 10 variables más importantes eran:
1. Income Before Tax (20% de la ganancia total de todos los splits)
2. Net Income (13% de la ganancia total)
3. Operating Income (7%)
4. Gross Profit (4%)
5. minority Interest (3%)
6. Stock based compensation (3%)
7. EBITDA (2.8%)
8. totalStockholdersEquity (2.6%)
9. operatingCashFlow (2.56)
10. researchAndDevelopmentExpenses (1.7)

Uno de de los principales problemas que se detectó es que 4 primeras variables, así como la 7tima, pertenecen todas al Income Statement, hablán sobre valores parecidos y hay mucha multicolinearidad (como era de esperar).

Con el fin de solucionar esto, se ha optado por, en vez de pasar las variable n valores absolutos, se han pasado ratios financieros, la función que genera todos estos ratios `generate_features()` se encuentra en myfuncs.py

## Model Build
### model_selection.ipynb
Una vez obtenidos todos los ratios, se han separado los datos en 2 partes. Una anterior a 2018 y otra posterior. La parte anterior, se ha utilizado para entrenar, se ha divido en train y test y se ha procedidos a probar varios modelos:
1. Un primer modelo naive, que asigna a todos los valores la media del target. Se hace esto con el objetivo de obetener el límite superior para el mse, por encima del cual no esta justificado ningún modelo. mse 5000
2. **regresion lineal**: mse 570
3. **regresión polinómica**: mse del orden de $10^{120}$
4. **SVM kernel lineal**: mse 566. Para valores altos de C (por encima de 0.2) el problema no converge si no se empieza a aumentar el numero de máximas iteraciondes, esto se debe a que el problema no es linealmente separable y el optimizador se atasca
5. **SVM con Radial Basis Functions**: mse de 567. Tarda demasiado en entrenar. 
6. **KNN**. Previa seleccion de los 5 features más importantes. mse de 636.
7. **Random Forest**: mse 428.3632211142772. Hace overfit
8. **Gradient Boosting**. mse 286

## Test
- Si se prueban los datos para datos futuros, se obtiene un mse de en torno a 16.000, una vez 